In [1]:
import glob
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imutils
from rasterio import features

# Create yolo labels

In [3]:
# label encoding
def label_encoding(x):
    labels = {'Disc':0, 'Cup':1}
    return labels[x]    
    
def extract_yolo(path, img_name, path_masks):
    
    if not os.path.exists(path):
        os.makedirs(path)
    
    j = 'Disc'
    
    for i in range(len(img_name)):
        mask_image = cv2.imread(path_masks + img_name[i] + '.bmp')
        mask_image = cv2.cvtColor(mask_image, cv2.COLOR_BGR2GRAY)
        
        mask_image = abs(255-mask_image)
        # Convert to binary image using thresholding
        mask_image = cv2.threshold(mask_image, 50, 255, cv2.THRESH_BINARY)[1]  # cup optic        
        
        cords = list(features.shapes(mask_image, mask=(mask_image >0)))[0][0]['coordinates'][0]
        label_line = str(label_encoding(j))+ ' ' + ' '.join([f'{int(cord[0])/mask_image.shape[0]} {int(cord[1])/mask_image.shape[1]}' for cord in cords])

        # Save bounding boxes to a file in YOLOv8 format
        with open(path + '/' + img_name[i] + '.txt', 'a') as f:
            f.write(label_line)

        # Remove duplicate
        with open(path + '/' + img_name[i] + '.txt', 'r') as input_file:
            lines = input_file.readlines()
            # Remove duplicates from the list of lines
            unique_lines = sorted(list(set(lines)))
            # Open the output file and write the unique lines to it
            with open(path + '/' + img_name[i] + '.txt', 'w') as output_file:
                output_file.writelines(unique_lines)

    return label_line



In [2]:
# Ler o nome de todas as imagens de treino e teste
def image_name(dir_path):
    img_name = []
    for image_path in glob.glob(dir_path + "/*"):
        file = image_path.split("\\")[-1][:-4]
        img_name.append(file)
    return img_name



img_name_train = image_name('C:/Users/debora.assis/Documents/REFUGE/train/images/')
img_name_val = image_name('C:/Users/debora.assis/Documents/REFUGE/valid/images/')
img_name_test = image_name('C:/Users/debora.assis/Documents/REFUGE/test/images/')



In [4]:
# pastas que estão localizadas as máscaras 
path_masks_train = 'C:/Users/debora.assis/Documents/REFUGE/masks/train/'
path_masks_val = 'C:/Users/debora.assis/Documents/REFUGE/masks/valid/'
path_masks_test = 'C:/Users/debora.assis/Documents/REFUGE/masks/test/'


# pastas que serão com as saídas das labels
path_output_train = 'C:/Users/debora.assis/Documents/REFUGE/train/labels/'
path_output_val = 'C:/Users/debora.assis/Documents/REFUGE/valid/labels/'
path_output_test = 'C:/Users/debora.assis/Documents/REFUGE/test/labels/'

yolo_train = extract_yolo(path_output_train, img_name_train, path_masks_train)
yolo_val = extract_yolo(path_output_val, img_name_val, path_masks_val)
yolo_test = extract_yolo(path_output_test, img_name_test, path_masks_test)